### 0. 라이브러리 임포트

In [133]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [234]:
import pandas as pd
import numpy as np
from glob import glob
import json
from tqdm.notebook import tqdm

import torch


### 1.데이터 로드 및 확인

In [145]:
train_data = pd.read_csv("open/train_data.csv")
test_data = pd.read_csv("open/test_data.csv")
sample_submission = pd.read_csv("open/sample_submission.csv")

In [4]:
train_data.head()

,id,text,summary
0,0,원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해...,원고가 주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 ...
1,1,수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 같은 그룹내 종합무역상사인 소외 ...,수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하기...
2,2,가등기담보권자가 제소전 화해조항에 따라 자기 명의로 소유권이전의 본등기를 경료한 후...,가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고 ...
3,3,가. 부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 뜻은 ...,부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 의미는 제1...
4,4,소득세법 제116조 제1항의 규정에 의하면 정부는 과세표준확정신고를 하여야 할 자에...,소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 할 자에 대해 당...


In [5]:
train_data.shape

(126632, 3)

In [6]:
test_data.head()

,id,text,summary
0,1,[경주=환경일보] 강광태 기자 = 경주시는 창의적 아이디어를 적극 수렴함으로써 소통...,생성요약문
1,2,"황영민 시장·군수·구청장 직접 제출… 경기도, 복지부·국회에 개선 요구 경기도가 장...",생성요약문
2,3,김대연 기자l saint-jj@hanmail.net 전북도의회는 올해 의회운영의 전...,생성요약문
3,4,대법원 2부(주심 안철상)는 스타 강사 우모씨를 상대로 인터넷 강의 업체 이투스교육...,생성요약문
4,5,"사설 보호무역 확산, G20회의 철저한 준비를 1월 수출이 월별 수출입 동향을 집계...",생성요약문


In [7]:
test_data.shape

(10000, 3)

In [8]:
sample_submission.head()

,id,summary
0,1,생성요약문
1,2,생성요약문
2,3,생성요약문
3,4,생성요약문
4,5,생성요약문


In [146]:
sample_submission.shape

(10000, 2)

### 데이터 전처리

In [147]:
#결측치 제거
nan_list = [27556, 34092, 34201, 66897, 133273, 216203]

train_data = train_data[~train_data["id"].isin(nan_list)].reset_index(drop = True)

In [148]:
val_data = train_data.iloc[-4000:, :]
train_data = train_data.iloc[:-4000, :]

In [14]:
train_data.head()

,id,text,summary,decoder_input,decoder_target
0,0,원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해...,원고가 주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 ...,sostoken 원고가 주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위...,원고가 주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 ...
1,1,수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 같은 그룹내 종합무역상사인 소외 ...,수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하기...,sostoken 수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수...,수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하기...
2,2,가등기담보권자가 제소전 화해조항에 따라 자기 명의로 소유권이전의 본등기를 경료한 후...,가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고 ...,sostoken 가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본...,가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고 ...
3,3,가. 부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 뜻은 ...,부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 의미는 제1...,sostoken 부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한...,부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 의미는 제1...
4,4,소득세법 제116조 제1항의 규정에 의하면 정부는 과세표준확정신고를 하여야 할 자에...,소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 할 자에 대해 당...,sostoken 소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 ...,소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 할 자에 대해 당...


In [149]:
test_data["decoder_input"] = test_data["summary"].apply(lambda x : "sostoken " + str(x))
test_data["decoder_target"] = test_data["summary"].apply(lambda x : str(x) + " eostoken")

encoder_input = test_data["text"]
decoder_input = test_data["decoder_input"]
decoder_output = test_data["decoder_target"]

In [47]:
test_data.head()

,id,text,summary,decoder_input,decoder_target
0,1,[경주=환경일보] 강광태 기자 = 경주시는 창의적 아이디어를 적극 수렴함으로써 소통...,생성요약문,sostoken 생성요약문,생성요약문 eostoken
1,2,"황영민 시장·군수·구청장 직접 제출… 경기도, 복지부·국회에 개선 요구 경기도가 장...",생성요약문,sostoken 생성요약문,생성요약문 eostoken
2,3,김대연 기자l saint-jj@hanmail.net 전북도의회는 올해 의회운영의 전...,생성요약문,sostoken 생성요약문,생성요약문 eostoken
3,4,대법원 2부(주심 안철상)는 스타 강사 우모씨를 상대로 인터넷 강의 업체 이투스교육...,생성요약문,sostoken 생성요약문,생성요약문 eostoken
4,5,"사설 보호무역 확산, G20회의 철저한 준비를 1월 수출이 월별 수출입 동향을 집계...",생성요약문,sostoken 생성요약문,생성요약문 eostoken


In [ ]:
for i in train_data.text[5:10]:
    print(i)
    print("------"*15)

가. 조세채권의 소멸시효를 규정하고 있는 국세기본법 제27조 제1항 소정의 국세의 징수를 목적으로 하는 권리라 함은 궁극적으로 국세징수의 실현만족을 얻는 일련의 권리를 말하는 것이므로, 여기에는 추상적으로 성립된 조세채권을 구체적으로 확정하는 국가의 기능인 부과권과 그 이행을 강제적으로 추구하는 권능인 징수권을 모두 포함하고 있다 할 것이므로 다른 특별한 규정이 없는 한 위 양자가 다같이 소멸시효의 대상이 된다.
------------------------------------------------------------------------------------------
버스운전사가 과속으로 운전하다가, 동 버스를 추월해 들어오는 택시와의 충돌을 피하기 위해 급히 핸들을 돌린 탓으로 좌측 4미터 아래의 하천으로 추락하여 승객 30명에게 상해를 입힌 경우, 그 사고발생의 동기가 뒤따라오던 택시와의 충돌을 면하기 위하여 핸들을 좌회한데 있다 하더라도 버스운전사로서는 부근의 지리적 상황을 감안하여 감속하였더라면 사고를 쉽게 피할 수 있었다고 보여지므로 이는 통상 발생할 수 있는 사고가 아닌, 자동차운수사업법 제31조 제5호 소정의 "중대한 교통사고"에 해당한다.
------------------------------------------------------------------------------------------
광업법 제5조, 제24조, 제26조, 제28조의 규정취지는 동일한 광구에서 어떤 광물에 관하여 광업권 설정이 된 경우에는 그 광물과 동일 광상에 부존하는 다른 광물에 대한 광업권설정은 이를 허가하지 아니한다는 것으로 볼 것이고 또 일단 설정된 광업권은 취소되지 않는 한 그 광업권의 존속기간동안 유효하게 존속하는 것이며 그 광업권 출원구역의 광물채굴이 경제적 가치가 없다는 이유만으로 당연히 소멸하는 것은 아니라 할 것이므로, 이건 광구에 설정된 규석광업권이 적법하게 취소된 바 없다면 그 규석과 동일한 모암에 함께 함유되어 있는 수정에 대하여 또 다

In [ ]:
for i,sent in enumerate(train_data.summary[5:10]):
    print(i,sent)
    print("------"*15)

0 국세기본법 제27조 제1항에 비추어 조세채권을 확정하는 국가의 기능인 부과권과 그 이행을 강제하는 징수권도 소멸시효의 대상이 되는 한편, 소멸시효는 객관적으로 권리가 발생하고 그 권리를 행사할 수 있는 때부터 진행되는 것이어서 권리를 행사할 수 없는 동안은 소멸시효는 진행할 수 없고, 사실상 그 권리의 존재나 권리행사 가능성에 대한 부지나 그 부지에 대한 과실유무 등은 시효진행에 영향을 미치지 않는다.
------------------------------------------------------------------------------------------
1 버스운전사의 과속으로 운전하다가  승객이 상해를 입힌 경우 버스운전사가 충돌을 면하기 위해 감속을 해도 사고를 쉽게 피할 수 있다고 보여지므로 중대한 교통사고에 해당한다.
------------------------------------------------------------------------------------------
2 광업법 제5조, 제24조, 제26조, 제28조는 동일한 광구에서 어떤 광물에 관해 광업권 설정이 된 경우 공일 광상에 부존하는 다른 광물에 대한 광업권설정을 허가하지 아니한다는 것이고 일단 설정된 광업권은 취소되지 않는 한 존속기간동안 유효하게 존속하는 것이며 출원구역의 광물채굴이 경제적가치가 없다는 이유로 소멸되는 것은 아니므로 그 광업권이 적법하게 취소된 바 없다면 또 다시 광업권설정을 허가한 것은 위법하다.
------------------------------------------------------------------------------------------
3 행정서사인 피고인이 여러 사람으로부터 계속 반복하여 등기신청 의뢰를 받고 등기신청에 필요한 매도증서, 위임장을 작성하고 수수료를 받았다면 사법서사법 제3조가 금하는 비사법서사의 사법서사업무행위에 해당된다.
------------------------------------------------

# train data 는 학습하기에 부적합 하다고 판단!!! 

In [86]:
import re

def preprocess_all(texts):
    #remove email & photo info & press, copyright
    preprocessed_text = []
    re_patterns = [
        r"\([^(]*?(뉴스|경제|일보|미디어|데일리|한겨례|타임즈|위키트리)\)",
        r"[가-힣]{0,4} (기자|선임기자|수습기자|특파원|객원기자|논설고문|통신원|연구소장) ",  # 이름 + 기자
        r"[가-힣]{1,}(뉴스|경제|일보|미디어|데일리|한겨례|타임|위키트리)",  # (... 연합뉴스) ..
        r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
        r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))",
        r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+",
        r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ",
        r"[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9%\'\"~\-\(\)\.,]"
    ]
    for text in texts:
        for re_pattern in re_patterns:
            text = re.sub(re_pattern, "", text).strip()
        if text:
            preprocessed_text.append(text)
            
    
    return preprocessed_text

In [263]:
train_data.summary[11]

'공소사실과 원심인정 사실이 이 사건 충돌사고의 원인을 피고인이 전방주시의무를 태만히 하고 앞차와의 안전거리를 충분히 확보하지 않은 것으로 보고 있다면 공소사실이 앞차가 감속하는 것을 뒤늦게 발견하여 이 건 사고가 야기되었다고 한 것을 원심이 앞차가 후진하기 위해 정차하고 있는 것을 너무 가까운 지점에서 발견하여 위 사고가 야기되었다고 인정했더라도 이 같은 차이는 지엽적이어서 근본적인 과실인정에 영향을 미친 것은 못되므로 이를 가리켜 불고불리의 원칙에 위배된다 볼 수 없다.'

In [233]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
#  Load Model and Tokenize
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")

# Encode Input Text
input_text = list(test_data['text'])
#input_text = preprocess_all(input_text) #전처리

In [ ]:
max_len = tokenizer.encode(input_text[0], return_tensors="pt").size()[1]
for i in range(1,len(input_text)):
    length = tokenizer.encode(input_text[i], return_tensors="pt").size()[1]
    if length > max_len:
        max_len = length

In [173]:
max_len #무지성 코드 , 이게 가장 길구나!!! 

1527

In [176]:
len(input_text[4083]) ##가장 길어서 에러난 거였다!!

3641

In [226]:
print("max:" , max([input_ids[i].size()[1] for i in range(len(input_text))]))
print("min:" , min([input_ids[i].size()[1] for i in range(len(input_text))]))

max: 1527
min: 3


In [264]:
summary = []

for idx, text in tqdm(enumerate(input_text)):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    #encoding 값이 1000보다 길면 요약 안하고 자기자신으로 처리
    if input_ids.size()[1] > 1000:
        summary.append(input_text[idx])
    else:
        # Generate Summary Text Ids
        summary_text_ids = model.generate(
             input_ids = input_ids,
             bos_token_id=model.config.bos_token_id,
             eos_token_id=model.config.eos_token_id,
             length_penalty=2.0,
             max_length=170,
             min_length=70,
             num_beams=4,
           )

        summary.append(tokenizer.decode(summary_text_ids[0] , skip_special_tokens=True))

In [276]:
#요약한 갯수 확인
len(summary)
cut_summary = summary.copy()

In [290]:
summary[9999]

'최근도 제주도감사위원회가 최근 버스준공영제에 대한 성과감사를 실시해 불합리한 문제점 35건을 확인하고 도지사에게 주의 처분과 제도개선을 요구한 가운데, 다른 지방에선 버스준공영제에 따른 과도한 재정 부담과 업체의 방만 경영, 채용 비리 등의 문제가 꾸준히 지적되고 있어 공공성과 재정투명성을 담보할 수 있는 강력한 장치가 도입돼야 할 때다.'

In [277]:
#요약문이 본문보다 긴 경우
for idx,text in tqdm(enumerate(input_text)):
    if len(text) < len(summary[idx]):
        cut_summary[idx] = text

In [279]:
sample_submission.summary = cut_summary

In [280]:
sample_submission.summary

0       경주시는 4월 30일까지 '2019년 경주시 상반기 행복 톡톡 아이디어 공모'를 통...
1       경차량 등록 후 주거지 관할 주민센터를 방문해 장애인차량 표지발급을 신청해야 하는 ...
2       전북도의회는 올해 의회운영의 전문성을 높이고, 지역현안에 보다 효율적으로 대처하기 ...
3       대법원 2부는 28일 인터넷 강의 제공업체 이투스교육이 수학 강사 우모씨를 상대로 ...
4       수출월별 수출입 동향에 따르면 수출은 32.8% 줄고 무역수지는 30억달러 적자를 ...
                              ...                        
9995    한국은행 금융통화위원회는 어제 기준금리를 2.75%에서 2.5%로 인하하여 작년 7...
9996    2012019 음성 전국사이클대회가 전국의 사이클 동호인과 엘리트 선수 등 900여...
9997    머니투데이가 금융과 관련된 5분 이내의 콘텐츠를 대상으로 총 상금 1000만원 규모...
9998    상표법은 등록상표가 일정한 사유에 해당하는 경우 별도로 마련한 상표등록의 무효심판절...
9999    최근도 제주도감사위원회가 최근 버스준공영제에 대한 성과감사를 실시해 불합리한 문제점...
Name: summary, Length: 10000, dtype: object

In [281]:
sample_submission.to_csv("last.csv", index = False)

In [287]:
sample_submission.isnull().sum()

id         0
summary    0
dtype: int64

In [17]:
val_data.head()

,id,text,summary,decoder_input,decoder_target
122628,122632,2일 광주 광산구 우산동 ‘2019광주세계수영선수권대회 선수촌’ 공개행사가 진행됐다...,지난 2일 조직위는 광주세계수영선수권대회 개막을 열흘 앞두고 광산구 우산동에 저비용...,sostoken 지난 2일 조직위는 광주세계수영선수권대회 개막을 열흘 앞두고 광산구...,지난 2일 조직위는 광주세계수영선수권대회 개막을 열흘 앞두고 광산구 우산동에 저비용...
122629,122633,국제수영선수권 대회를 앞둔 광주 곳곳이 불법주정차로 몸살을 앓고 있다. 사진은 지난...,광주 구청은 국제수영선수권 대회를 앞둔 광주 곳곳에 불법 주정차들이 보행자들의 통행...,sostoken 광주 구청은 국제수영선수권 대회를 앞둔 광주 곳곳에 불법 주정차들이...,광주 구청은 국제수영선수권 대회를 앞둔 광주 곳곳에 불법 주정차들이 보행자들의 통행...
122630,122634,전국학교비정규직노조 총파업으로 광주·전남지역 300여개 학교에서 단체급식이 중단된 ...,학교비정규직노조가 총파업에 들어가 오는 3일 광주지역 초·중·고교에서는 학교 조리실...,sostoken 학교비정규직노조가 총파업에 들어가 오는 3일 광주지역 초·중·고교에...,학교비정규직노조가 총파업에 들어가 오는 3일 광주지역 초·중·고교에서는 학교 조리실...
122631,122635,"하이주 “정글숲을 지나서 가자, 엉금 엉금 기어서 가자” 점점 더워지는 날씨, 아이...","광주에 위치한 하이주는 동물들과 교감하는 체감형 테마카페로 긴코너구리, 은여우 등을...",sostoken 광주에 위치한 하이주는 동물들과 교감하는 체감형 테마카페로 긴코너구...,"광주에 위치한 하이주는 동물들과 교감하는 체감형 테마카페로 긴코너구리, 은여우 등을..."
122632,122636,광주시체육회와 한국도로공사 스포츠단이 업무협약을 가졌다. 광주체육회 제공 광주광역시...,지난 3일 광주시체육회와 한국도로공사 스포츠단이 ‘광주체육회-도로공사 스포츠단 간...,sostoken 지난 3일 광주시체육회와 한국도로공사 스포츠단이 ‘광주체육회-도로...,지난 3일 광주시체육회와 한국도로공사 스포츠단이 ‘광주체육회-도로공사 스포츠단 간...


### 결국은 포로로였다.
- 전처리도 큰 의미가 없었고, fine-tunning도 데이터가 안좋아서 더 안좋아졌다. 
- task에 맞게 사전학습된 모델을 가져와서 쓰는게 좋았다.

In [ ]:
from pororo import Pororo
import pandas as pd
from tqdm import tqdm

if __name__ == "__main__":
    model = Pororo(task = "summarization", model = "abstractive", lang="ko")

    data = pd.read_csv("../dataset/test_data.csv")
    texts = data['text'].to_list()

    summ = []
    for text in tqdm(texts):
        summ.append(model(text))

    submission = pd.DataFrame()
    submission['id'] = data['id']
    submission['summary'] = summ

    submission.to_csv("abstractive.csv",index=False)